<a href="https://colab.research.google.com/github/lajaaiti/Alto_MVC_POO/blob/main/GPT2_Elvis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>GPT2 Elvis</h1>

In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.5 MB/s 
     |████████████████████████████████| 120 kB 61.8 MB/s 
     |████████████████████████████████| 6.6 MB 43.5 MB/s 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os
import sys

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [3]:
# data import
data = pd.read_csv('dfElvis.csv')
data.head()

,Titre,Lyric
0,Can't Help Falling In Love,"Wise men say, only fools rush in\nBut I can't ..."
1,Suspicious Minds,We're caught in a trap\nI can't walk out\nBeca...
2,Always On My Mind,"Maybe I didn't treat you,\nQuite as good as I ..."
3,Love Me Tender,"Love me tender, love me sweet,\nNever let me g..."
4,My Way,And now the end is near\nAnd so I face the fin...


In [4]:
# data train et data test
data_train = data[:int(len(data)*0.8)]
data_test = data[int(len(data)*0.8):]

In [5]:
class SongLyrics(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in data_train['Lyric']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))               
        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]
    
dataset = SongLyrics(data_train['Lyric'], truncate=True, gpt2_type="gpt2", max_length=980)  

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [7]:
def collate_batch(batch):
    '''
    input: List[(label, sentence)]
    '''
    
    for lyric in batch:
        processed_text = torch.tensor(tkn_text_pipeline(words), dtype=torch.int64)
        if processed_text.shape[0] == 0:  # skip empty sentences
            continue
        categories.append(category)
        label_list.append(label)
        text_list.append(processed_text)
        seq_len.append(processed_text.shape[0])
    if len(text_list) != 0:
        label_list = torch.tensor(label_list, dtype=torch.int64)
        text_list = pad_sequence(text_list, padding_value=PAD_IDX)
        seq_len = torch.tensor(seq_len, dtype=torch.int64)
        categories = torch.tensor(categories, dtype=torch.int64)  
    return categories.to(device), label_list.to(device), text_list.to(device), seq_len.to(device)

In [8]:
def train(
    dataset, model, tokenizer,
    batch_size=32, epochs=10, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [9]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().cpu().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().cpu().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(data_test):
  generated_lyrics = []
  for i in tqdm(range(len(data[:200]))):
    x = generate(model.to(device), tokenizer, data['Lyric'][i], entry_count=3)
    generated_lyrics.append(x)
  return generated_lyrics

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#Run the functions to generate the lyrics

generated_lyrics = text_generation(data)

100%|██████████| 200/200 [08:10<00:00,  2.45s/it]


In [10]:
from itertools import product
for i, j in product(range(5), range(3)):
    print(generated_lyrics[i][j][-200:])
print([generated_lyrics[j][i][-200:] for j in range(5) for i in range(3) ])

 life too
For I can't help falling in love with you

For I can't help falling in love with you

Or fall in love with your mother

Shall I stay? Would it be a sin

If I can't help, falling<|endoftext|>
 help falling in love with you

For I can't help falling in love with you

See how she walks in dark in the little house
Like a river flows, surely to the sea

You see she walks in bright<|endoftext|>
oo
For I can't help falling in love with you

For I can't help falling in love with you

Darling in love with you, then let's take a gamble

And let's have our days, our nights, our days,<|endoftext|>
don't believe a word I say?

We're caught in a trap
I can't walk out
Because I love you too much baby

Why can't you see

What you're doing to me

When you don't believe a word I say?

We<|endoftext|>
don't believe a word I say?

We're caught in a trap
I can't walk out
Because I love you too much baby

Why can't you see

What you're doing to me

When you don't believe a word I say?

We<|endofte

In [11]:
#Loop to keep only generated text and add it as a new column in the dataframe
my_generations=[]

for i in range(len(generated_lyrics)):
  a = data_test['Lyric'][i].split()[-50:] #Get the matching string we want (30 words)
  b = ' '.join(a)
  c = ' '.join(generated_lyrics[i]) #Get all that comes after the matching string
  my_generations.append(c.split(b)[-1])


# Add the generated text as a new column in the dataframe
# Length of values (200) does not match length of index (150)
data_test['Generated_lyrics'] = my_generations


#Finish the sentences when there is a point, remove after that
final=[]

for i in range(len(my_generations)):
  to_remove = my_generations['Generated_lyrics'][i].split('.')[-1]
  final.append(my_generations['Generated_lyrics'][i].replace(to_remove,''))

my_generations['Generated_lyrics'] = final

KeyError: ignored

In [ ]:
#Using BLEU score to compare the real sentences with the generated ones
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]

for i in range(len(test_set)):
  reference = test_set['True_end_lyrics'][i]
  candidate = test_set['Generated_lyrics'][i]
  scores.append(sentence_bleu(reference, candidate))

statistics.mean(scores)

KeyError: 'Generated_lyrics'

In [ ]:
model = train(dataset, model, tokenizer, batch_size=128, epochs=15, lr=1e-5,)